# OpenFEMA API Tutorial: Part 3 - Paging to Get Data
  
## Quick Summary
- This tutorial demonstrates the use of a "paging" technique to retrieve data when more than 10,000 records are found - *it is not a Python language tutorial*
- The OpenFEMA API parameters \\$skip and \\$top are used in conjunction with the record count to iterate through the data
- A [Final Working Example](#Final-Working-Example) (at the end of this document) demonstrates the paging technique in Python (our GitHub repository contains examples in other languages)
- The next tutorial will cover Getting Dataset Updates

## Overview
In previous tutorials we demonstrated the basic use of the OpenFEMA API and parameters to alter the content and delivery of  the data - the fields to return, the sort order, and *what* records to return based on criteria. Other parameters affect data delivery or *how* the data is returned - file format, file names, and the number of records to return at one time. Because OpenFEMA limits the maximum number of records returned per call to 10,000 (the default return count is 1,000 if \\$top is not explicitly set), we need to issue multiple calls using some of these parameters. 
 
The purpose of this notebook is to demonstrate how to retrieve more than 10,000 found records using a technique called "paging" that utilizes the \\$skip parameter, $top parameter, and the "count" value. The examples are presented using Python 3, but it should be easy to translate them to almost any programming language.

See the [OpenFEMA API Documentation](https://www.fema.gov/about/openfema/api) for details on the API parameters.

See the [OpenFEMA Code Samples GitHub Repository](https://github.com/FEMA/openfema-samples) for paging examples in various languages.

## IMPORTANT NOTE: Page no More!
In February 2024 OpenFEMA released a new API feature - an argument called \\$allrecords.This query argument forces all records to be returned as part of a download rather than adhering to the maximum return limit. It is an attempt to reduce the difficulty in downloading very large sets of data. Currently, any query resulting in more records than specified by \\$top will require the "paging" technique described in this tutorial to iterate through results to capture all of the data. The $allrecords=true query string overrides this behavior. This argument accepts a boolean value: true will return all records, false (the default) will not return all the records.

Note that this will not have the effect of retrieving and displaying all records __within a browser__ window. If used in a browser, it must be used in conjunction with a file download as in: [https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries?\\$format=csv&\\$filename=DDSV2_all.csv&\\$allrecords=true](https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries?$format=csv&$filename=DDSV2_all.csv&$allrecords=true)
 
<div class="alert alert-block alert-danger">
    <b>Warning:</b> This is being released as a BETA feature. It essentially elmiminates the need for "paging" to get data. If we find it negatively impacts OpenFEMA system performance or it is abused, it may be removed. 
</div>

__We will keep this notebook as the paging techniques descirbed are still valid. Those who have existing paging code may want to keep it as it works well on most datasets. Computers with low memory can also benefit from this technique - if the data to return is larger than the available memory, iterating in chunks and saving to disk is an option.__

The following example results in the same data as the final working example in this notebook, however it uses the new \\$allrecords argument to get all the data:

In [3]:
# declare a URL handling module
import urllib.request
import json

# define URL for the Disaster Declarations Summaries endpoint
baseUrl = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries"

# define a query using parameters 
select = "?$select=disasterNumber,declarationDate,declarationTitle,state"    # leave this parameter out if you want all fields
filter = "&$filter=state%20ne%20%27FL%27"                                    # for purposes of example, exclude Florida
orderby = "&$orderby=id"                                                     # order unimportant to me, so use id
format = "&$format=json"    
other = "&$allrecords=true&$metadata=off"                                    # return all records, suppress metadata

# issue api call
request = urllib.request.urlopen(baseUrl + select + filter + orderby + format + other)
result = request.read()
    
# The data is already returned in a json format. There is no need to decode and load as a JSON object.
#   If you want to begin working with and manipulating the JSON, import the json library and load with
#   something like: jsonData = json.loads(result.decode())

# save to file
outFile = open("dds_output.json", "w")
outFile.write(str(result,'utf-8'))
outFile.close()

# lets re-open the file and see if we got the number of records we expected
inFile = open("dds_output.json", "r")
my_data = json.load(inFile)
print(str(len(my_data['DisasterDeclarationsSummaries'])) + " records in file")
inFile.close()

62934 records in file


## Paging Through Data
For performance reasons, 1,000 records are returned by default with a maximum value of 10,000 per API endpoint call. If more than 10,000 records are found, it will be necessary to page through the data to retrieve every record. The metadata header returned as part of the dataset JSON response will only display the full record count if the \\$inlinecount parameter is used—otherwise, it will have a value of 0. Computer code containing a loop is written to continue making API calls, incrementing the \\$skip parameter accordingly until the number of records retrieved equals the total record count.

<div class="alert alert-block alert-warning">
    <b>Note:</b> Several of the GitHub code examples download CSV files. It is recommended that results be downloaded in a JSON format. This format is native to the OpenFEMA data store, as such data need not be converted by the server thereby improving download performance.
</div>

### Defining and Calling the API Query
Define the endpoint and any parameters. As explained in Parts 1 and 2 of the tutorial, the process to call an API endpoint is straightforward - declare a library or module that can facilitate requests to web resources, define the URL/API endpoint, specify parameters, and issue the API call.

<div class="alert alert-block alert-info">
    <b>Tip:</b> 
    <ul>
    <li>If you need the full, unfiltered file, use one of the file download links on the dataset page.
    <li>For very large files, try to limit the data with the \$filter and \$select parameters. This is optional of course.</li>
    <li>Specify a sort order to ensure that the proper records are returned with each call. If the sort is unimportant, use the id field.</li>
    </ul>
</div>

The previous tutorials used the FemaWebDisasterDeclarations endpoint. This tutorial will use the DisasterDeclarationsSummaries dataset as it contains many more records, allowing us to filter and still find more than 10,000 records.

In [1]:
# declare a URL handling module
import urllib.request
import json

# define URL for the Disaster Declarations Summaries endpoint
baseUrl = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries"

# define a query using parameters 
select = "?$select=disasterNumber,declarationDate,declarationTitle,state"    # leave this parameter out if you want all fields
filter = "&$filter=state%20ne%20%27FL%27"                                    # for purposes of example, exclude Florida

Let's make an initial call using the \\$inlinecount parameter. We want to know how many records match our criteria. This will tell us how many times we must call the endpoint. Why not just include this parameter in our main request for data - the one in which we call multiple times? As explained in Part 2 of the tutorial, \\$count forces the OpenFEMA server to count all the matching records *for each call*. If working with a large dataset (such as NFIP policies that contains more than 61 million records) where you must issue thousands of calls to retrieve all the data, including $count in every call will significantly reduce performance.

For this initial call, limit it to 1 record, 1 field, and we don't care about the sort or the return format. Rather than define these parameter values in variables and combine, we will just add them to our URL and filter.

<div class="alert alert-block alert-warning">
    <b>Note:</b> Because we are ignoring the \$select parameter above, we have to add back in our base URL/query separator - the question mark (?). The order in which the parameters are combined is not important.
</div>

In [2]:
# Return 1 record with your criteria to get total record count. Specify only 1
#   column to reduce amount of data returned. Need inlinecount to get record count. 
request = urllib.request.urlopen(baseUrl + "?$count=true&$select=id&$top=1" + filter )

# actually read the data
result = request.read()

# transform to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# display the metadata object
print(json.dumps(jsonData['metadata'], indent=2))

{
  "skip": 0,
  "filter": "state ne 'FL'",
  "orderby": "",
  "select": null,
  "rundate": "2023-06-01T22:22:36.018Z",
  "DeprecationInformation": {
    "depDate": null,
    "depApiMessage": null,
    "depNewURL": null,
    "depWebMessage": null
  },
  "top": 1,
  "format": "json",
  "metadata": true,
  "entityname": "DisasterDeclarationsSummaries",
  "version": "v2",
  "url": "/api/open/v2/DisasterDeclarationsSummaries?$inlinecount=allpages&$select=id&$top=1&$filter=state%20ne%20%27FL%27",
  "count": 62037
}


Now that we have a total count of records matching our criteria, we can calculate the number of times we must issue the call to return all of it. We will retrieve the maximum number of records we can (10,000) so we can minimize the number of calls we must make. So, divide by our count by our record limit. Round any fractional amount up or the last call will be missed. We will use the "math" library to help with this.

<div class="alert alert-block alert-warning">
    <b>Note:</b> Because the default number of records returned is 1,000, we have to explicitly define the $top parameter.
</div>

In [3]:
import math

# number of records we want to return with each call
top = 10000

# calculate the number of calls we will need to get all of our data (using the maximum of 1000)
recCount = jsonData['metadata']['count']
loopNum = math.ceil(recCount / top)

# print amounts
print("For " + str(recCount) + " records, we will need to issue " + str(loopNum) + " calls")

For 62037 records, we will need to issue 7 calls


Before we define our loop and begin to issue calls, let's take care of a few more items:

 - Define additional parameters:
   - \\$orderby - order is unimportant for the final result in this example, but we should specify one - use the id field
   - \\$top - explicitly defined above - if we don't do this, only 1,000 records will be returned per call
   - \\$metadata - let's turn off, otherwise the calls will take longer and we will have to skip over the metadata returned. Since we will be using a jsona format, turning metadata off is optional because specifying the jsona format will suppress the metadata automatically. We will include it in this example for clarity.
   - \\$format - use jsona instead of json to simplify 
 - Open a file so that we can save our data. While this is optional (you could save the data in a variable and then proceed with some sort of manipulation or analysis), the most common use case is to save the data so it can be easily retrieved and used later.

<div class="alert alert-block alert-warning">
    <b>Note:</b> Earlier, we defined the call return type to be jsona. Why? If we use the json format, every call will result in a top level json object (named after the dataset endpoint) containing an array of records. The top level object will have to be skipped for each iteration - all we want are the records. While this is not hard to do in Python, specifying jsona gives us just an array of records without the top level object.
    
A side effect of this technique is that the final file (of just saved, jsona records) will not be recognized as a valid json file for future use (because it really is a json array file). This can be overcome by adding the high level object back into the file one time.
</div>

<div class="alert alert-block alert-info">
    <b>Tip:</b> A csv format may be specified if desired. The code in these examples will have to change slightly. The code samples in the OpenFEMA GitHub repository has examples. 
    
There is a tradeoff with regards to specifying a csv format. Less data must be transmitted because the format is not as verbose, but the OpenFEMA server must work to convert the data from json to csv - potentially increasing the download time. It is also possible to convert the format after it has been downloaded.
</div>

In [7]:
orderby = "&$orderby=id"     # order unimportant to me, so use id field
limit = "&$top=10000"        # needed otherwise the default of 1000 will apply
format = "&$format=jsona"    # let's use an array of json objects - easier
other = "&$metadata=off"     # not needed as jsona suppresses metadata - including for clarity


# Initialize our file. Only doing this because of the type of file wanted. See the loop below.
#   The root json entity is usually the name of the dataset, but you can use any name.
outFile = open("dds_output.json", "w")
outFile.write('{"disasterdeclarationssummaries":[');

Now we are ready to make subsequent calls until all our data has been fetched. With each iteration, skip the records that have already been fetched.

 - Compose a request, skipping those already fetched
 - Continue issuing calls until we have reached our loop count
 - Determine if we are on the last call to terminate the json object if necessary
 
<div class="alert alert-block alert-info">
    <b>Tip:</b> An alternative to saving the data after each call is to accumulate the results in a variable and then save it all at once. While this can be faster, it may not be possible due to memory limitations for extremely large datasets.
</div>
 

In [8]:
# Loop and call the API endpoint changing the record start each iteration. 
skip = 0
i = 0
while (i < loopNum):
    # By default, data is returned as a json object, the data set name being the root element. Unless
    #   you extract records as you process, you will end up with 1 distinct json object for EVERY 
    #   call/iteration. An alternative is to return the data as jsona (an array of json objects) with 
    #   no root element - just a bracket at the start and end. This is easier to manipulate.
    request = urllib.request.urlopen(baseUrl + select + filter + orderby + limit + format + other + "&$skip=" + str(skip))
    result = request.read()
    
    # The data is already returned in a json format. There is no need to decode and load as a JSON object.
    #   If you want to begin working with and manipulating the json, import the json library and load with
    #   something like: jsonData = json.loads(result.decode())

    # Append results to file, trimming off first and last JSONA brackets, adding comma except for last call,
    #   AND root element terminating array bracket and brace to end unless on last call. The goal here is to 
    #   create a valid json file that contains ALL the records. This can be done differently.
    if (i == (loopNum - 1)):
        # on the last so terminate the single JSON object
        outFile.write(str(result[1:-1],'utf-8') + "]}")
    else:
        outFile.write(str(result[1:-1],'utf-8') + ",")

    # increment the loop counter and skip value
    i+=1
    skip = i * top

    print("Iteration " + str(i) + " done")

outFile.close()

Iteration 1 done
Iteration 2 done
Iteration 3 done
Iteration 4 done
Iteration 5 done
Iteration 6 done
Iteration 7 done


At this point, all the data has been retrieved and saved. We can verify that all the records were retrieved by opening the file, loading the data, and counting the number of records.

In [9]:
# let's re-open the file and see if we got the number of records we expected
inFile = open("dds_output.json", "r")
my_data = json.load(inFile)
print(str(len(my_data['disasterdeclarationssummaries'])) + " records in file")
inFile.close()


62037 records in file


## Modifications for CSV Format
The following illustrates saving data in a csv format. We changed the \\$format parameter and removed the code to make the result a proper json file. It is necessary to check whether we are on the first pull to ensure we output the field headers only once. 

In [10]:
orderby = "&$orderby=id"     # order unimportant to me, so use id field
limit = "&$top=10000"        # needed otherwise the default of 1000 will apply
format = "&$format=csv"      # let's use csv as our output type
other = "&$metadata=off"     # not needed as csv suppresses metadata - including for clarity


# Initialize our file. Only doing this because of the type of file wanted. See the loop below.
#   The root json entity is usually the name of the dataset, but you can use any name.
outFile = open("dds_output.csv", "w")


# Loop and call the API endpoint changing the record start each iteration. The metadata is being
#   suppressed as we no longer need it.
skip = 0
i = 0
while (i < loopNum):
    # issue call, decode the data, and save to a file
    request = urllib.request.urlopen(baseUrl + select + filter + orderby + limit + format + other + "&$skip=" + str(skip))
    result = request.read()
    csvData = result.decode('utf-8')
    
    # avoid writing the header/fieldnames every time
    if (i == 0):
        # on the first record, so write full output that includes field headers
        outFile.write(csvData)
    else:
        # split off the first row
        outFile.write(csvData.split("\n",1)[1])
    
    # increment the loop counter and skip value
    i+=1
    skip = i * top

    print("Iteration " + str(i) + " done")

outFile.close()

Iteration 1 done
Iteration 2 done
Iteration 3 done
Iteration 4 done
Iteration 5 done
Iteration 6 done
Iteration 7 done


## Alternative 1 - Saving to Variable and Writing to File Once
In the preceding examples, we have written each chunk of 10,000 records to a file as we go. An easier and definetly faster technique is to accumulate each 10,000 chunk in a variable and write it out once. If you have more memory available than the records returned will consume, then this will work fine. Unfortunately, many of the datasets have variable length records so it is not possible to determine up front how much memory you need. The danger is that your code runs for an hour, runs out of memory on the last chunk, and your data is lost when you get an "out of memory" error. 

There exist only a handful of OpenFEMA datasets for which this may happen: Valid Registrations, Large Disasters, NFIP Policies, NFIP Claims, and IPAWS. Just be mindful that this can happen if you use this technique. You can also write code to monitor memory usage and write to disk if it gets low, but this seems like extra effort.

The following is an example of the JSONA example above, but storing data in a variable. 

In [14]:
# declare a URL handling module
import urllib.request
import json
import math

# define URL for the Disaster Declarations Summaries endpoint
baseUrl = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries"

# define a query using parameters 
select = "?$select=disasterNumber,declarationDate,declarationTitle,state"    # leave this parameter out if you want all fields
filter = "&$filter=state%20ne%20%27FL%27"                                    # for purposes of example, exclude Florida
orderby = "&$orderby=id"     # order unimportant to me, so use id
limit = "&$top=10000"        # needed otherwise the default of 1000 will apply
format = "&$format=jsona"    # lets use an array of json objects - easier
other = "&$metadata=off"     # not needed as jsona suppresses metadata - including for clarity

# number of records we want to return with each call
top = 10000

# Return 1 record with your criteria to get total record count. Specifying only 1
#   column here to reduce amount of data returned. Need inlinecount to get record count. 
request = urllib.request.urlopen(baseUrl + "?$count=true&$select=id&$top=1" + filter )

# actually read the data
result = request.read()

# transform to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# calculate the number of calls we will need to get all of our data (using the maximum of 1000)
recCount = jsonData['metadata']['count']
loopNum = math.ceil(recCount / top)

# Loop and call the API endpoint changing the record start each iteration. The metadata is being
#   suppressed as we no longer need it.
cumulativeResults = ""
skip = 0
i = 0
while (i < loopNum):
    # By default data is returned as a json object, the data set name being the root element. Unless
    #   you extract records as you process, you will end up with 1 distinct json object for EVERY 
    #   call/iteration. An alternative is to return the data as jsona (an array of json objects) with 
    #   no root element - just a bracket at the start and end. This is easier to manipulate. Another 
    #   would be to use JSONL (no brackets to remove) and replace the line feed with a comma.
    request = urllib.request.urlopen(baseUrl + select + filter + orderby + limit + format + other + "&$skip=" + str(skip))
    result = request.read()
    
    # The data is already returned in a json format. There is no need to decode and load as a JSON object.
    #   If you want to begin working with and manipulating the JSON, import the json library and load with
    #   something like: jsonData = json.loads(result.decode())

    # Append results to a variable, trimming off first and last jsona brackets, adding a comma to seperate 
    #   each object/record. We could turn results into a list and append to a list variable, but this is as easy.
    cumulativeResults = cumulativeResults + str(result[1:-1],'utf-8') + ","
    
    # increment the loop counter and skip value
    i+=1
    skip = i * top

    print("Iteration " + str(i) + " done")
    
# Save our variable to a file. We must add the root element, remove the trailing comma,
#   and add the terminating array bracket and brace to end. The goal here is to create
#   a valid json file that contains ALL the records.
outFile = open("dds_output_alt.json", "w")
outFile.write('{"disasterdeclarationssummaries":[');
outFile.write(cumulativeResults[:-1] + "]}")
outFile.close()

# lets re-open the file and see if we got the number of records we expected
inFile = open("dds_output_alt.json", "r")
my_data = json.load(inFile)
print(str(len(my_data['disasterdeclarationssummaries'])) + " records in file")
inFile.close()

Iteration 1 done
Iteration 2 done
Iteration 3 done
Iteration 4 done
Iteration 5 done
Iteration 6 done
Iteration 7 done
62037 records in file


## Alternative 2 - Saving to Variable and Writing to File Once
Here is an alternative to the above. The differences include:
 - The external "requests" library is used rather than "urllib.request". This external library is more robust and most of the examples found online use it. It adds a dependency, however.
 - It uses the json format, not jsona. There is no need to wrap the results in a element.
 - It uses a looping construct that eliminates the need to first get a count of records to return. Less code/complexity, but you lose a record count check.
 - It wraps the api call in a more generic function to retrieve data.
 - This version has a little bit of error handling
 - Output is a json array - no top level object

In [30]:
# alternative function to read data using paging - all at once in memory
import requests

# create a function that will call the OpenFEMA API and return filtered data 
def fetch_api_data(endpointVersion, endpointName, filter, select, recordLimit=10000, baseUrl="https://www.fema.gov/api/open/v"):
    skipRecords = 0
    allData = []
    
    while True:
        print("Fetching data starting from record " + str(skipRecords) + "...") 

        # construct final url (Note that if any of these items are missing, the call will fail - 
        #    could create a separate, generic OpenFEMA API URL string builder)
        url = baseUrl + endpointVersion + "/" + endpointName + "?$filter=" + filter + "&$select=" + select + \
                "&$top=" + str(recordLimit) + "&$skip=" + str(skipRecords) + "&$format=json&$metadata=off"
        
        try:
            response = requests.get(url) 

            if response.status_code == 200: # Indicates the request has succeeded
                data = response.json().get(endpointName, [])
                if not data:
                    break  # Break the loop if no more data is returned

                allData = allData + data
                
                skipRecords += recordLimit
            else:
                print(f"Failed to fetch data: HTTP {response.status_code}")
                break  # Break the loop on failure
            
        except requests.exceptions.RequestException as e:
            print("Request error.")
            raise SystemExit(e)
        
    print("Done.")
    return allData


# This uses the first function that pages to get data
select = 'disasterNumber,declarationDate,declarationTitle,state'
filter = 'state%20ne%20%27FL%27'
finalData = fetch_api_data(endpointVersion='2', endpointName='DisasterDeclarationsSummaries',filter=filter,select=select) 

# essentially, this saves a json array - no top level object
with open("dds_output_alt2.json", "w") as fp:
    json.dump(finalData, fp)

# lets read to see if correct number of records written
with open("dds_output_alt2.json", "r") as fp:
    my_data = json.load(fp)
    
print(str(len(my_data)) + " records in file")


Fetching data starting from record 0...
Fetching data starting from record 10000...
Fetching data starting from record 20000...
Fetching data starting from record 30000...
Fetching data starting from record 40000...
Fetching data starting from record 50000...
Fetching data starting from record 60000...
Fetching data starting from record 70000...
Done.
62934 records in file


## Final Working Example
Although contrived, this final example applies many of the parameters discussed in this tutorial.  Examples in other languages can be found in the [OpenFEMA Samples on GitHub](https://github.com/FEMA/openfema-samples) repository.

<div class="alert alert-block alert-warning">
    <b>Note:</b> This is not meant to be a Python language tutorial. The point is to show the use and combination of OpenFEMA API parameters. There are other, more Pythonic ways that this can be done. If you are writing production quality code, it is recommended that you follow industry best practices - evaluate returned values, add error handling, add logging, proper object cleanup, build for resilience by adding retries if failure, etc.
</div>

In [15]:
# declare a URL handling module
import urllib.request
import json
import math

# define URL for the Disaster Declarations Summaries endpoint
baseUrl = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries"

# define a query using parameters 
select = "?$select=disasterNumber,declarationDate,declarationTitle,state"    # leave this parameter out if you want all fields
filter = "&$filter=state%20ne%20%27FL%27"                                    # for purposes of example, exclude Florida
orderby = "&$orderby=id"     # order unimportant to me, so use id
limit = "&$top=10000"        # needed otherwise the default of 1000 will apply
format = "&$format=jsona"    # lets use an array of json objects - easier
other = "&$metadata=off"     # not needed as jsona suppresses metadata - including for clarity

# number of records we want to return with each call
top = 10000

# Return 1 record with your criteria to get total record count. Specifying only 1
#   column here to reduce amount of data returned. Need inlinecount to get record count. 
request = urllib.request.urlopen(baseUrl + "?$count=true&$select=id&$top=1" + filter )

# actually read the data
result = request.read()

# transform to Python dictionary
jsonData = json.loads(result.decode('utf-8'))

# calculate the number of calls we will need to get all of our data (using the maximum of 1000)
recCount = jsonData['metadata']['count']
loopNum = math.ceil(recCount / top)

# Initialize our file. Only doing this because of the type of file wanted. See the loop below.
#   The root json entity is usually the name of the dataset, but you can use any name.
outFile = open("dds_output.json", "w")
outFile.write('{"disasterdeclarationssummaries":[');

# Loop and call the API endpoint changing the record start each iteration. The metadata is being
#   suppressed as we no longer need it.
skip = 0
i = 0
while (i < loopNum):
    # By default data is returned as a json object, the data set name being the root element. Unless
    #   you extract records as you process, you will end up with 1 distinct json object for EVERY 
    #   call/iteration. An alternative is to return the data as jsona (an array of json objects) with 
    #   no root element - just a bracket at the start and end. This is easier to manipulate.
    request = urllib.request.urlopen(baseUrl + select + filter + orderby + limit + format + other + "&$skip=" + str(skip))
    result = request.read()
    
    # The data is already returned in a json format. There is no need to decode and load as a JSON object.
    #   If you want to begin working with and manipulating the JSON, import the json library and load with
    #   something like: jsonData = json.loads(result.decode())

    # Append results to file, trimming off first and last jsona brackets, adding comma except for last call,
    #   AND root element terminating array bracket and brace to end unless on last call. The goal here is to 
    #   create a valid json file that contains ALL the records. This can be done differently.
    if (i == (loopNum - 1)):
        # on the last so terminate the single JSON object
        outFile.write(str(result[1:-1],'utf-8') + "]}")
    else:
        outFile.write(str(result[1:-1],'utf-8') + ",")

    # increment the loop counter and skip value
    i+=1
    skip = i * top

outFile.close()

# lets re-open the file and see if we got the number of records we expected
inFile = open("dds_output.json", "r")
my_data = json.load(inFile)
print(str(len(my_data['disasterdeclarationssummaries'])) + " records in file")
inFile.close()

62037 records in file


## Where to Go Next
OpenFEMA API_Tutorial_Part_4_GettingDatasetUpdates

## Other Resources
- [OpenFEMA Homepage](https://www.fema.gov/open)
- [OpenFEMA API Documentation](https://www.fema.gov/about/openfema/api)
- [OpenFEMA Samples on GitHub](https://github.com/FEMA/openfema-samples)